In [1]:
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import pickle

In [2]:
item_genre_dict = pickle.load(open('./item_genre_dict.pkl'))
rdf = pickle.load(open('./rdf.pkl'))

In [3]:
item_list = rdf['item_id'].unique()
user_list = rdf['user_id'].unique()
print('item num = ' + str(len(item_list)))
print('user num = ' + str(len(user_list)))

item num = 31025
user num = 102612


In [4]:
# count the number for each genre and sort
import operator
genre_count = dict()
for l in item_genre_dict:
    for g in item_genre_dict[l]:
        if not g in genre_count:
            genre_count[g] = 1
        else:
            genre_count[g] += 1

genre_count_sorted = sorted(genre_count.items(), key=operator.itemgetter(1), reverse=True)
genre_count_sorted

[(u'Restaurants', 31025),
 (u'Food', 5598),
 (u'Nightlife', 5111),
 (u'Bars', 4958),
 (u'American (Traditional)', 4340),
 (u'Sandwiches', 3574),
 (u'Pizza', 3131),
 (u'Breakfast & Brunch', 3127),
 (u'American (New)', 3022),
 (u'Fast Food', 2992),
 (u'Mexican', 2854),
 (u'Italian', 2761),
 (u'Burgers', 2732),
 (u'Chinese', 2467),
 (u'Japanese', 1703),
 (u'Seafood', 1606),
 (u'Salad', 1470),
 (u'Cafes', 1447),
 (u'Sushi Bars', 1430),
 (u'Asian Fusion', 1217),
 (u'Chicken Wings', 1189),
 (u'Coffee & Tea', 1173),
 (u'Sports Bars', 1093),
 (u'Event Planning & Services', 1093),
 (u'Steakhouses', 1072),
 (u'Mediterranean', 1007),
 (u'Barbeque', 995),
 (u'Delis', 940),
 (u'Canadian (New)', 904),
 (u'Pubs', 891),
 (u'Diners', 883),
 (u'Thai', 881),
 (u'Caterers', 807),
 (u'Desserts', 786),
 (u'Indian', 779),
 (u'Specialty Food', 756),
 (u'Vegetarian', 730),
 (u'Bakeries', 673),
 (u'Wine Bars', 644),
 (u'Vietnamese', 617),
 (u'Middle Eastern', 600),
 (u'Greek', 590),
 (u'Cocktail Bars', 574),
 (

In [5]:
key_genre = ['American (New)', 'Chinese', 'Japanese', 'Italian']

# get the key_genre->item_list dict
key_genre_item = dict()
for k in key_genre:
    key_genre_item[k] = list()
for item in item_genre_dict:
    for g in item_genre_dict[item]:
        if g in key_genre:
            key_genre_item[g].append(item)
            
# collect all the items with key genres
key_item = list()
for genre in key_genre_item:
    key_item = key_item + key_genre_item[genre]
key_item_set = set(key_item)

# remove the non-key genre items in rdf
item_set = set(item_list)
nonkey_item_set = item_set - key_item_set
for item in nonkey_item_set:
    rdf.drop(rdf.index[rdf['item_id'] == item], inplace=True)

In [6]:
import copy
rdf_copy = copy.copy(rdf)

In [24]:
rdf = copy.copy(rdf_copy)

In [29]:
# iteratively remove items and users with less than 5 reviews

rdf['user_freq'] = rdf.groupby('user_id')['user_id'].transform('count')
rdf.drop(rdf.index[rdf['user_freq'] <= 12], inplace=True)
rdf['item_freq'] = rdf.groupby('item_id')['item_id'].transform('count')
rdf.drop(rdf.index[rdf['item_freq'] <= 12], inplace=True)
rdf['user_freq'] = rdf.groupby('user_id')['user_id'].transform('count')
rdf['user_id'].value_counts()

CxDOIDnH8gp9KXzpBHJYXw    515
cMEtAiW60I5wE_vLfTxoJQ    388
bLbSNkLggFnqwNNzzq-Ijw    380
d_TBs6J3twMy9GChqUEXkg    366
PKEzKWv_FktMm2mGPjwd0Q    284
U4INQZOPSUaj8hMjLlZ3KA    269
DK57YibC5ShBmqQl97CKog    225
C2C0GPKvzWWnP57Os9eQ0w    224
kjeX2RXvW7RhBbD2QLd5jA    222
UYcmGbelzRa0Q6JqzLoguw    221
ELcQDlf69kb-ihJfxZyL0A    207
S9Jw00eZHVj5_0sOM_C5Rg    205
iDlkZO2iILS8Jwfdy7DP9A    204
V-BbqKqO8anwplGRx9Q5aQ    201
1kNsEAhGU8d8xugMuXJGFA    199
2e5V6M4GNufEnbGJpVdCjw    197
tH0uKD-vNwMoEc3Xk3Cbdg    194
EiP1OFgs-XGcKZux0OKWIA    192
dIIKEfOgo0KqUfGQvGikPg    186
JaqcCU3nxReTW2cBLHounA    180
8DEyKVyplnOcSKx39vatbg    178
N3oNEwh0qgPqPP3Em6wJXw    176
QJI9OSEn6ujRCtrX06vs1w    171
n86B7IkbU20AkxlFX_5aew    164
RBZ_kMjowV0t6_nv2UKaDQ    162
M9rRM6Eo5YbKLKMG5QiIPA    161
pwQ8E-hbDUJe3qtKt_IBBg    160
qewG3X2O4X6JKskxyyqFwQ    154
oeAhRa8yFa9jtrhaHnOyxQ    145
SCo1UBoeN3bhRMkSYuiX1A    145
                         ... 
ntuYgYgQEMj2-skkch2qQw     13
9eRRQL3JwekCLFPY25tO2A     13
wL-eQ6ZPYj

In [30]:
item_list = rdf['item_id'].unique()
user_list = rdf['user_id'].unique()
print('item num = ' + str(len(item_list)))
print('user num = ' + str(len(user_list)))

item num = 4420
user num = 8263


In [31]:
# get the user and item str id->int id dict
i = 0
user_id_dict = dict()
for u in user_list:
    if not u in user_id_dict:
        user_id_dict[u] = i
        i += 1
j = 0
item_id_dict = dict()
for i in item_list:
    if not i in item_id_dict:
        item_id_dict[i] = j
        j += 1

In [33]:
print('sparsity: ' + str(len(rdf) * 1.0 / (len(user_list) * len(item_list))))

sparsity: 0.00579684391468


In [34]:
# get the df of train, vali, and test set
rdf.reset_index(inplace=True, drop=True)
train_df = rdf.copy()
vali_df = rdf.copy()
test_df = rdf.copy()

train_ratio = 0.6
vali_ratio = 0.2
test_ratio = 0.2
num_all = len(rdf)
vali_idx = []
test_idx = []

test_vali_idx = []
i = 0
num_user = len(user_list)
for u in user_list:
    u_idx = train_df.index[train_df['user_id'] == u]
    idx_len = len(u_idx)
    test_len = int(idx_len * (test_ratio + vali_ratio))
    if test_len == 0:
        test_len = 1
    tmp = np.random.choice(u_idx, size=test_len, replace=False)
    test_vali_idx += tmp.tolist()
    i += 1
    if i % 5000 == 0:
        print(str(i) + '/' + str(num_user))

# tmp = (np.random.choice(range(num_all), size=(test_len+vali_len), replace=False)).tolist()
test_len = int(len(test_vali_idx) * test_ratio / (test_ratio + vali_ratio))
vali_len = int(len(test_vali_idx) - test_len)
test_idx = (np.random.choice(test_vali_idx, size=test_len, replace=False)).tolist()
vali_idx = (np.random.choice(test_vali_idx, size=vali_len, replace=False)).tolist()

test_set = set(test_idx)
vali_set = set(vali_idx)
train_set = set(range(num_all)) - test_set - vali_set
train_idx = list(train_set)
train_df.drop((test_idx + vali_idx), axis=0, inplace=True)
test_df.drop((train_idx + vali_idx), axis=0, inplace=True)
vali_df.drop((train_idx + test_idx), axis=0, inplace=True)

5000/8263


In [35]:
rdf.drop(columns=['rating'], inplace=True)
train_df.drop(columns=['rating'], inplace=True)
test_df.drop(columns=['rating'], inplace=True)
vali_df.drop(columns=['rating'], inplace=True)

In [36]:
# get the matrix of train, vali and test set

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
vali_df.reset_index(drop=True, inplace=True)
rdf.reset_index(drop=True, inplace=True)
train = np.zeros((len(user_list), len(item_list)))
test = np.zeros((len(user_list), len(item_list)))
vali = np.zeros((len(user_list), len(item_list)))
for r in range(len(train_df)):
    train[user_id_dict[train_df.at[r, 'user_id']], item_id_dict[train_df.at[r, 'item_id']]] = 1.0
for r in range(len(test_df)):
    test[user_id_dict[test_df.at[r, 'user_id']], item_id_dict[test_df.at[r, 'item_id']]] = 1.0
for r in range(len(vali_df)):
    vali[user_id_dict[vali_df.at[r, 'user_id']], item_id_dict[vali_df.at[r, 'item_id']]] = 1.0

In [37]:
# get the user int id-> str id list, and the same for item 
item_list = item_id_dict.keys()
item_idd_list = list()
for i in range(len(item_list)):
    item_idd_list.append('')
for item in item_id_dict:
    item_idd_list[item_id_dict[item]] = item

user_list = user_id_dict.keys()
user_idd_list = list()
for i in range(len(user_list)):
    user_idd_list.append('')
for user in user_id_dict:
    user_idd_list[user_id_dict[user]] = user
    
# get the item int id->genres list
item_idd_genre_list = list()
for i in range(len(item_idd_list)):
    item_idd_genre_list.append(item_genre_dict[item_idd_list[i]])

In [38]:
train_df.drop('user_freq', axis=1, inplace=True)
train_df.drop('item_freq', axis=1, inplace=True)
vali_df.drop('user_freq', axis=1, inplace=True)
vali_df.drop('item_freq', axis=1, inplace=True)
test_df.drop('user_freq', axis=1, inplace=True)
test_df.drop('item_freq', axis=1, inplace=True)
rdf.drop('user_freq', axis=1, inplace=True)
rdf.drop('item_freq', axis=1, inplace=True)

In [39]:
# get df for rdf, train, vali, test with int id for user and item
import copy
rating_df = copy.copy(rdf)
for i in range(len(rdf)):
    rating_df.at[i, 'user_id'] = user_id_dict[rating_df.at[i, 'user_id']]
    rating_df.at[i, 'item_id'] = item_id_dict[rating_df.at[i, 'item_id']]

training_df = copy.copy(train_df)
for i in range(len(training_df)):
    training_df.at[i, 'user_id'] = user_id_dict[training_df.at[i, 'user_id']]
    training_df.at[i, 'item_id'] = item_id_dict[training_df.at[i, 'item_id']]

valiing_df = copy.copy(vali_df)
for i in range(len(valiing_df)):
    valiing_df.at[i, 'user_id'] = user_id_dict[valiing_df.at[i, 'user_id']]
    valiing_df.at[i, 'item_id'] = item_id_dict[valiing_df.at[i, 'item_id']]

testing_df = copy.copy(test_df)
for i in range(len(testing_df)):
    testing_df.at[i, 'user_id'] = user_id_dict[testing_df.at[i, 'user_id']]
    testing_df.at[i, 'item_id'] = item_id_dict[testing_df.at[i, 'item_id']]

In [40]:
# generate the rating list for each key genre, get the genre->ratings dict
rdf.reset_index(drop=True, inplace=True)
key_genre_rating = dict()
for k in key_genre:
    key_genre_rating[k] = 0.0
for r in range(len(rdf)):
    item = rdf.at[r, 'item_id']
    gl = item_genre_dict[item]
    for k in key_genre:
        if k in gl:
            key_genre_rating[k] += 1.0

# get the item int id->genres list
genre_item_vector = dict()
for k in key_genre:
    genre_item_vector[k] = np.zeros((1, len(item_list)))
for i in range(len(item_idd_genre_list)):
    genre_list = item_idd_genre_list[i]
    for g in genre_list:
        if g in key_genre:
            genre_item_vector[g][0, i] = 1.0

In [41]:
with open("item_genre_dict.pkl", "wb") as f:
    pickle.dump(item_genre_dict, f, pickle.HIGHEST_PROTOCOL)
with open("genre_item_vector.pkl", "wb") as f:
    pickle.dump(genre_item_vector, f, pickle.HIGHEST_PROTOCOL)
with open("key_genre.pkl", "wb") as f:
    pickle.dump(key_genre, f, pickle.HIGHEST_PROTOCOL)
with open("user_id_dict.pkl", "wb") as f:
    pickle.dump(user_id_dict, f, pickle.HIGHEST_PROTOCOL)
with open("item_id_dict.pkl", "wb") as f:
    pickle.dump(item_id_dict, f, pickle.HIGHEST_PROTOCOL)
# with open("rdf.pkl", "wb") as f:
#     pickle.dump(rdf, f, pickle.HIGHEST_PROTOCOL)
with open("rating_df.pkl", "wb") as f:
    pickle.dump(rating_df, f, pickle.HIGHEST_PROTOCOL)
with open("training_df.pkl", "wb") as f:
    pickle.dump(training_df, f, pickle.HIGHEST_PROTOCOL)
with open("valiing_df.pkl", "wb") as f:
    pickle.dump(valiing_df, f, pickle.HIGHEST_PROTOCOL)
with open("testing_df.pkl", "wb") as f:
    pickle.dump(testing_df, f, pickle.HIGHEST_PROTOCOL)
with open("item_idd_genre_list.pkl", "wb") as f:
    pickle.dump(item_idd_genre_list, f, pickle.HIGHEST_PROTOCOL)
with open("item_idd_list.pkl", "wb") as f:
    pickle.dump(item_idd_list, f, pickle.HIGHEST_PROTOCOL)
with open("user_idd_list.pkl", "wb") as f:
    pickle.dump(user_idd_list, f, pickle.HIGHEST_PROTOCOL)
with open("key_genre_rating.pkl", "wb") as f:
    pickle.dump(key_genre_rating, f, pickle.HIGHEST_PROTOCOL)
    
with open("train.mat", "wb") as f:
    np.save(f, train)
with open("test.mat", "wb") as f:
    np.save(f, test)
with open("vali.mat", "wb") as f:
    np.save(f, vali)

In [42]:
# count the number for each genre and sort
import pickle
from operator import itemgetter
# item_list = pickle.load(open('./rdf.pkl'))['item_id'].unique()
# item_genre_dict = pickle.load(open('./item_genre_dict.pkl'))
# key_genre = pickle.load(open('./key_genre.pkl'))

genre_count = dict()
for i in item_list:
    gl = item_genre_dict[i]
    for g in gl:
        if g in key_genre:
            if not g in genre_count:
                genre_count[g] = 1
            else:
                genre_count[g] += 1

with open("genre_count.pkl", "wb") as f:
    pickle.dump(genre_count, f, pickle.HIGHEST_PROTOCOL)
                
genre_count_sorted = sorted(genre_count.items(), key=itemgetter(1), reverse=True)
genre_count_sorted

[(u'American (New)', 1610),
 (u'Italian', 1055),
 (u'Chinese', 984),
 (u'Japanese', 946)]

In [43]:
import numpy as np
import pickle
import copy as copy

item_idd_genre_list = np.array(item_idd_genre_list)


mask = 1.0 * (train > 0)
user_genre_count = list()
for u in range(train.shape[0]):
    temp_genre_count = copy.copy(genre_count)
    mask_u = mask[u, :]
    gll = item_idd_genre_list[mask_u == 1.0]
    for gl in gll:
        for g in gl:
            if g in key_genre:
                temp_genre_count[g] -= 1
    user_genre_count.append(temp_genre_count)
with open("user_genre_count.pkl", "wb") as f:
    pickle.dump(user_genre_count, f, pickle.HIGHEST_PROTOCOL)

In [44]:
genre_avg_like = dict()
for k in key_genre:
    genre_avg_like[k] = key_genre_rating[k] * 1.0 / genre_count[k]

In [45]:
genre_avg_like_sorted = sorted(genre_avg_like.items(), key=itemgetter(1), reverse=True)
genre_avg_like_sorted

[('American (New)', 56.844099378881985),
 ('Japanese', 48.10570824524313),
 ('Italian', 44.0132701421801),
 ('Chinese', 37.326219512195124)]